# PyWorkbench and PyMechanical

In [ ]:
import os
import json
import pyvista as pv
from ansys.workbench.core import launch_workbench
from ansys.mechanical.core import launch_mechanical

In [ ]:
# launch Workbench service on the local machine
wb = launch_workbench(client_workdir='C:/Users/fli/demo')

In [ ]:
# upload a geometry file to the server
wb.upload_file_from_example_repo("2pipes.agdb", "2pipes")

In [ ]:
# run a Workbench script to create a mechanical system and load geometry
system_name = wb.run_script_file('geometry.wbjn')

In [ ]:
# start PyMechanical service for the system and create a PyMechanical client
pymech_port = wb.start_mechanical_server(system_name=system_name)
mechanical = launch_mechanical(start_instance=False, ip='localhost', port=pymech_port)
print(mechanical.project_directory)

In [ ]:
# run a Mechanical python script via PyMechanical to mesh/solve the model
with open (os.path.join(wb.client_workdir, "solve.py")) as sf:
    mech_script = sf.read()
print(mechanical.run_python_script(mech_script))

In [ ]:
# download a couple of output files from the solver directory
mechanical.download("*solve.out", target_dir=wb.client_workdir)
mechanical.download("*deformation.png", target_dir=wb.client_workdir)

In [ ]:
# display the content of the solver output
with open(os.path.join(wb.client_workdir, "solve.out"), "r") as f:
    print(f.read())

In [ ]:
# plot the deformation result generated by the solver
pl = pv.Plotter()
pl.add_background_image(os.path.join(wb.client_workdir, "deformation.png"))
pl.show(jupyter_backend='static')

In [18]:
# shutdown the Workbench service
wb.exit()